#Multi-Task Learning using AraBert for Offensive Language Detection

Notebook used in the OSACT4 - shared task on Offensive language detection (LREC 2020)
http://edinburghnlp.inf.ed.ac.uk/workshops/OSACT4/ 

Task Paper: https://www.aclweb.org/anthology/2020.osact-1.16/ 

Authors: Marc Djandji, Fady Baly, Wissam Antoun

In [0]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
##install arabert if not already done
!git clone -b macrof1 https://github.com/WissamAntoun/bert

import bert.tokenization as tokenization
import bert.modeling as modeling
import bert.optimization as optimization
from tqdm import tqdm
import numpy as np

from google.colab import auth, drive
auth.authenticate_user()

Cloning into 'bert'...
remote: Enumerating objects: 415, done.
remote: Total 415 (delta 0), reused 0 (delta 0), pack-reused 415
Receiving objects: 100% (415/415), 420.19 KiB | 20.01 MiB/s, done.
Resolving deltas: 100% (234/234), done.



In [0]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)

b'Tesla P4'


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/'My Drive'/best_MTL_model/best_model.zip ./

In [0]:
!unzip /content/best_model.zip 

Archive:  /content/best_model.zip
   creating: content/best_model/
  inflating: content/best_model/events.out.tfevents.1582011537.9f8209d16eb2  
  inflating: content/best_model/checkpoint  
  inflating: content/best_model/model.ckpt-1200.meta  
  inflating: content/best_model/graph.pbtxt  
  inflating: content/best_model/model.ckpt-1200.index  
  inflating: content/best_model/model.ckpt-1200.data-00000-of-00001  


In [0]:
!cp -r /content/content/best_model/model.ckpt-1200.index /content/Output_Dir/1250000/
!cp -r /content/content/best_model/model.ckpt-1200.meta /content/Output_Dir/1250000/
!cp -r /content/content/best_model/model.ckpt-1200.data-00000-of-00001 /content/Output_Dir/1250000/

In [0]:
!mv /content/Output_Dir/1250000/model.ckpt-1200.data-00000-of-00001 /content/Output_Dir/1250000/model.ckpt-0.data-00000-of-00001
!mv /content/Output_Dir/1250000/model.ckpt-1200.index /content/Output_Dir/1250000/model.ckpt-0.index
!mv /content/Output_Dir/1250000/model.ckpt-1200.meta /content/Output_Dir/1250000/model.ckpt-0.meta

In [0]:
!cp -r /content/drive/'My Drive'/Bert/arabert.zip /content/
# # !cp -r /content/drive/'My Drive'/Trained_bert/Output_Dir.zip /content

In [0]:
# # !unzip Output_Dir.zip
!unzip arabert.zip

In [0]:
# ##use tf_arabert model, change path accordingly
BERT_VOCAB= ''
BERT_INIT_CHKPNT = ''
BERT_CONFIG = ''

In [0]:
tokenization.validate_case_matches_checkpoint(True, BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(vocab_file=BERT_VOCAB,
                                       do_lower_case=True)

### Data Preprocessing


In [0]:
!pip install pyarabic
!gsutil cp gs://bert_pretrain4/FarasaSegmenterJar.jar ./
!pip install emoji

def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null    #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version
install_java()
!pip install py4j
!pkill "java"

     |████████████████████████████████| 102kB 12.1MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.6-cp36-none-any.whl size=106208 sha256=7a3c3dcca57c1ec79e80a2db3f72f7234463bb8df9375c8d0a7ecce5730d95ee
  Stored in directory: /root/.cache/pip/wheels/34/b5/2d/668d567e8c2b6f10309dbfaba5bfef6ea0b1c0f9f6fb37078f
Successfully built pyarabic
Copying gs://bert_pretrain4/FarasaSegmenterJar.jar...
/ [1 files][ 13.0 MiB/ 13.0 MiB]                                                
Operation completed over 1 objects/13.0 MiB.                                     
     |████████████████████████████████| 51kB 9.4MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=e40cb86eb868f95d6934ce361da628a09b871a1cfefc23ddd22fc6d6fc234f8b
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu1

In [0]:
# get preprocessing script
# !gsutil cp gs://bert_pretrain4/preprocess_wiss.py ./
!cp -r /content/drive/'My Drive'/Bert/preprocess_wiss.py ./
from preprocess_wiss import preprocess
# download data
!gsutil cp -r gs://osact4/data ./

import emoji
def remove_emoji(text):
    """
    :param text: enter a string of words.
    :return: removes emojis from the input string
    """
    return emoji.get_emoji_regexp().sub(u'', text)

eastern_to_western = {"٠":"0","١":"1","٢":"2","٣":"3","٤":"4","٥":"5","٦":"6","٧":"7","٨":"8","٩":"9","٪":"%","_":" ","ڤ":"ف","|":" "}
trans_string = str.maketrans(eastern_to_western)

Copying gs://osact4/data/OSACT2020-sharedTask-dev.txt...
Copying gs://osact4/data/OSACT2020-sharedTask-train.txt...
/ [2 files][  1.5 MiB/  1.5 MiB]                                                
Operation completed over 2 objects/1.5 MiB.                                      


In [0]:
!cp /content/drive/'My Drive'/Bert/OSACT2020-sharedTask-test-tweets.txt /content/

In [0]:
f = open('/content/OSACT2020-sharedTask-test-tweets.txt', 'r', encoding='utf-8')
lines = f.readlines()
test = pd.DataFrame(np.array(lines).reshape(-1,1),columns=['text'])

train = pd.read_csv("./data/OSACT2020-sharedTask-train.txt",sep="\t",header=None)
train.columns = ["text","offensive","hate_speech"]

eval_ = pd.read_csv("./data/OSACT2020-sharedTask-dev.txt",sep="\t",header=None)
eval_.columns =  ["text","offensive","hate_speech"]

train["text"] = train["text"].apply(lambda x : remove_emoji(x))
train["text"] = train["text"].apply(lambda x : x.replace("<LF>"," "))
train["text"] = train["text"].apply(lambda x : x.translate(trans_string))
train["text"] = train["text"].apply(lambda x : preprocess(x, True))
train["text"] = train["text"].apply(lambda x : x.replace("\\"," "))
train["offensive"] = train["offensive"].apply(lambda x : x.replace("NOT_OFF", '0'))
train["offensive"] = train["offensive"].apply(lambda x : x.replace("OFF", '1'))
train["hate_speech"] = train["hate_speech"].apply(lambda x : x.replace("NOT_HS", '0'))
train["hate_speech"] = train["hate_speech"].apply(lambda x : x.replace("HS", '1'))

eval_["text"] = eval_["text"].apply(lambda x : remove_emoji(x))
eval_["text"] = eval_["text"].apply(lambda x : x.replace("<LF>"," "))
eval_["text"] = eval_["text"].apply(lambda x : x.translate(trans_string))
eval_["text"] = eval_["text"].apply(lambda x : preprocess(x, True))
eval_["text"] = eval_["text"].apply(lambda x : x.replace("\\"," "))
eval_["offensive"] = eval_["offensive"].apply(lambda x : x.replace("NOT_OFF", '0'))
eval_["offensive"] = eval_["offensive"].apply(lambda x : x.replace("OFF", '1'))
eval_["hate_speech"] = eval_["hate_speech"].apply(lambda x : x.replace("NOT_HS", '0'))
eval_["hate_speech"] = eval_["hate_speech"].apply(lambda x : x.replace("HS", '1'))

In [0]:
test["text"] = test["text"].apply(lambda x : remove_emoji(x))
test["text"] = test["text"].apply(lambda x : x.replace("<LF>"," "))
test["text"] = test["text"].apply(lambda x : x.translate(trans_string))
test["text"] = test["text"].apply(lambda x : preprocess(x, True))
test["text"] = test["text"].apply(lambda x : x.replace("\\"," "))

In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMNS = ['offensive', 'hate_speech']

In [0]:
x_train = train
x_val = eval_
x_test = test

In [0]:
x_train_w_eval = pd.concat([x_train, x_val], axis=0)

In [0]:
x_train_w_eval.shape

(7839, 3)

#### Downlsampling

In [0]:
x_train['hate_speech'] = x_train['hate_speech'].astype(np.uint8)
x_train['offensive'] = x_train['offensive'].astype(np.uint8)

In [0]:
hate_idx = x_train[x_train.hate_speech == 1].index
off_idx = x_train[x_train.offensive == 1].index
non_hate_indices = x_train[x_train.hate_speech == 0].index
non_off_indices = x_train[x_train.offensive == 0].index

In [0]:
hate_idx

Int64Index([   2,   40,   66,   74,   85,   94,  182,  183,  200,  212,
            ...
            6670, 6697, 6713, 6732, 6735, 6745, 6753, 6773, 6791, 6806],
           dtype='int64', length=350)

In [0]:
diff = non_hate_indices.shape[0]-non_off_indices.shape[0]
nans = np.ones((diff,))*100000
non_hate_indices = np.asarray(non_hate_indices)
non_off_indices =np.concatenate((non_off_indices,nans),axis=0)

matches = []

for i in range(non_hate_indices.shape[0]):
    for j in range(non_hate_indices.shape[0]):
        if non_off_indices[j]== non_hate_indices[i]:
            matches.append(non_off_indices[j])

In [0]:
n_hate = len(hate_idx)
n_offense = len(off_idx)
n_neither = len(matches)

from prettytable import PrettyTable
    
tab = PrettyTable()

tab.field_names = ["","Train Offensive", 'Train Hate', 'Train Neither']
tab.add_row(["count", n_offense, n_hate, n_neither])
print(tab)

+-------+-----------------+------------+---------------+
|       | Train Offensive | Train Hate | Train Neither |
+-------+-----------------+------------+---------------+
| count |       1371      |    350     |      5468     |
+-------+-----------------+------------+---------------+


In [0]:
int(n_neither/1.5)

3645

In [0]:

n_hate = len(hate_idx)
n_offense = len(off_idx)
random_indices = np.random.choice(matches, int(n_neither/1.5), replace=False)

under_sample_indices = np.concatenate([hate_idx, off_idx, random_indices])
under_sampled = x_train.loc[under_sample_indices]

hate_data = x_train.loc[hate_idx] 

under_sampled = under_sampled.sample(frac=1).reset_index(drop=True)

# balanced = pd.concat([under_sampled, hate_data], axis=0)


In [0]:
x_train = under_sampled.sample(frac=1).reset_index(drop=True)

In [0]:
tot = len(under_sampled.values)
percentage_off = np.count_nonzero((under_sampled.values)[:,1] == 1)/tot
percentage_hate = np.count_nonzero((under_sampled.values)[:,2] == 1)/tot
percentage_non = 1-(percentage_off+ percentage_hate)
tab = PrettyTable()

tab.field_names = ["","Test Offensive", 'Test Hate', 'Test Neither']
tab.add_row(["Percentage", np.round(percentage_off,4)*100, np.round(percentage_hate,4)*100, np.round(percentage_non,4)*100])
print(tab)

+------------+----------------+-----------+--------------------+
|            | Test Offensive | Test Hate |    Test Neither    |
+------------+----------------+-----------+--------------------+
| Percentage |     32.07      |   13.05   | 54.879999999999995 |
+------------+----------------+-----------+--------------------+


In [0]:
x_train.head()

,text,offensive,hate_speech
0,[مستخدم] يعني هياخدوا دوري ال+ أبطال السن +ه د...,0,0
1,[مستخدم] مبروك يا دوك يا مبدع و+ عقبال ال+ ملي...,0,0
2,[مستخدم] يا عبري +ة يا أيتها ال+ عاهر +ة فك +ي...,1,1
3,يا بسم +ة ال+ صبح يا ف+ ألي و+ يا أملي يا نغم ...,0,0
4,RT [مستخدم] : صباح ال+ زفت علي +كم [مستخدم] يا...,1,0


In [0]:
balanced.head()

,text,offensive,hate_speech
0,RT [مستخدم] : يافاتنة كل ال+ مدن يا أرض ال+ نق...,0,0
1,أي يا حظ ، أي يا سفر ، أي يا فلوس ، أي ياجواز ...,1,0
2,أنا حاول +ت كتير أتصور ميرور سيلفي و+ ب+ يطلع ...,1,0
3,RT [مستخدم] : أقولك يا إيمان و+ لا يا نان +ا و...,0,0
4,#عبدالرحمن ال+ خشت +ي اللهم يا سيدي و+ يا خالق...,0,0


#### Class weighing


In [0]:
from sklearn.utils import class_weight
import numpy as np

labels_off= x_train['offensive'].values
labels_hate= x_train['hate_speech'].values
class_weights_off = class_weight.compute_class_weight('balanced',
                                                 np.unique(labels_off),
                                                 labels_off)
class_weights_hate = class_weight.compute_class_weight('balanced',
                                                 np.unique(labels_hate),
                                                 labels_hate)

In [0]:
class_weights_off

array([0.62536576, 2.49416484])

In [0]:
class_weights_hate

array([0.52696872, 9.77      ])

### Start model Components

In [0]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [0]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = ''
        text_a = row[0]
        if labels_available:
            labels = row[1:]
        else:
            labels = [0,0]
            print(text_a, labels)
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [0]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [0]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    batches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([2], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

#### Creating the MTL model

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings, class_weights_off = 1,
                 class_weights_Noff=1, class_weights_hate = 1, class_weights_Nhate=1 ):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)


    output_layer = model.get_pooled_output()
    


    hidden_size = output_layer.shape[-1].value
    weights = {}
    biases = {}
    # creating weights and biases per task (lable)

    for task in range(num_labels):
      
      key = 'task_{}'.format(task)
      weight_name = "output_weights_{}".format(task)
      bias_name = "output_bias_{}".format(task)
      
      weights[key] = tf.get_variable(
          weight_name, [2, hidden_size],
          initializer = tf.truncated_normal_initializer(stddev=0.02))
      
      biases[key] = tf.get_variable(
          bias_name, 
          [2], initializer=tf.zeros_initializer())


    # class_weights = tf.constant([[1.0, 2.0]])
    # deduce weights for batch samples based on their true label

    class_weights_off = tf.constant([[class_weights_Noff, class_weights_off]])
    class_weights_hate = tf.constant([[class_weights_Nhate, class_weights_hate]])

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        
        logits = []
        probabilities = []
        per_example_loss = []
        log_probs =[]
        labels_hot= []
        losses=[]

        for task in range(num_labels):
          key = 'task_{}'.format(task)

          logits.append(tf.matmul(output_layer, weights[key], transpose_b=True))
          logits[task] = tf.nn.bias_add(logits[task], biases[key])

          
          

          probabilities.append(tf.nn.softmax(logits[task], axis=-1))

          labels = tf.cast(labels, tf.int32) # labels shape: (batch_size, n_labels) (32,2)
          labels_hot.append(tf.one_hot(labels[:,task], depth=num_labels, dtype=tf.int32))
          
          tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))

          labels_hot[task] = tf.cast(labels_hot[task], tf.float32)


          ''' for weighted loss function '''
          # if task ==0:
          per_example_loss.append(tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits[task], labels=labels_hot[task] ) ))
            # per_example_loss[task] = per_example_loss[task] * class_weights_off
          # else:
            # per_example_loss.append(tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits[task], labels = labels_hot[task] ) ))
            # per_example_loss[task] = per_example_loss[task] * class_weights_hate
          # per_example_loss.append(tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits[task], labels=labels_hot[task]) ))

        loss = tf.math.reduce_sum(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
           tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            # tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)


        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):
                
                labels_hot = []
                for task in range(len(LABEL_COLUMNS)):

                  label_ids = tf.cast(label_ids, tf.int32) # labels shape: (batch_size, n_labels) (32,2)
                  labels_hot.append(tf.one_hot(label_ids[:,task], depth=num_labels, dtype=tf.int32))
                  # 0 non-offense, non-hate
                  # 1 offense, hate


                label_cols = ['non-offense','offense', 'non-hate', 'hate']

                non_offense = probabilities[0][:,0]
                offense = probabilities[0][:,1]
                non_hate = probabilities[1][:,0]
                hate = probabilities[1][:,1]

                proba = [non_offense, offense, non_hate, hate]
                labels_separate = [labels_hot[0][:,0], labels_hot[0][:,1], labels_hot[1][:,0], labels_hot[1][:,1]]
                # metrics change to auc of every class
                eval_dict = {}
                 
                for j, (label_name, logits) in enumerate(zip(label_cols, proba)):

                    logits = tf.math.round(logits)

                    current_f1, update_op_f1 = tf.contrib.metrics.f1_score(labels_separate[j], logits)  
                    current_accuracy, update_op_accuracy = tf.compat.v1.metrics.accuracy(labels_separate[j], logits)

                    
                    eval_dict[label_name + '_f1'] = (current_f1, update_op_f1)  # (current_auc, update_op_auc)
                    eval_dict[label_name + '_accuracy'] = (current_accuracy, update_op_accuracy)


                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict



            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops = eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities_offense": probabilities[0], 'probabilities_hate': probabilities[1]},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

### Training Preps

In [0]:
tokenizer = tokenization.FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=False)

MAX_SEQ_LENGTH = 256
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 200
SAVE_SUMMARY_STEPS = 200

!mkdir working
train_file = os.path.join('./working', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

train_examples = create_examples(x_train)
file_based_convert_examples_to_features(train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)

eval_file = os.path.join('./working', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

test_file = os.path.join('./working', "test.tf_record")
if not os.path.exists(test_file):
    open(test_file, 'w').close()

test_examples = create_examples(x_test)
file_based_convert_examples_to_features(test_examples, MAX_SEQ_LENGTH, tokenizer, test_file)

mkdir: cannot create directory ‘working’: File exists


In [0]:
x_val.head()

,text,offensive,hate_speech
0,في حاج +ات مينفعش نلفت نظرك +وا لي +ها زى ال+ ...,0,0
1,: و+ عيون تنادي +نا تحايل في +نا و+ نقول يا عي...,0,0
2,يا بلاد +ي يا أم ال+ بلاد يا بلاد +ي ب+ حبك يا...,0,0
3,: يا رب يا قوي يا معين مدني ب+ ال+ قو +ة و+ ال...,0,0
4,: رحم ك+ الله يا صدام يا بطل و+ مقدام .,0,0


In [0]:
x_test.head()

,text,offensive,hate_speech
0,اما أنت تقعد طول عمر ك+ لا مبدا و+ لا رأي ثابت...,0.0,0.0
1,ب+ تخاف نسوان ك+ يزعل +وا و+ لا أي اه يا هلفوت...,0.0,0.0
2,: يا عسانى نبقى يا عمري حبايب و+ حب +نا يكبر م...,0.0,0.0
3,: باقي ال+ بيان و+ ينو ما شفن +ه يا برهان و+ ر...,0.0,0.0
4,اللهم أنت ال+ شافي ال+ معافي اشفي +ه و+ جميع م...,0.0,0.0


In [0]:
x_test['offensive'] = np.zeros((x_test.shape[0]))
x_test['hate_speech'] = np.zeros((x_test.shape[0]))

In [0]:
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 6839
INFO:tensorflow:  Batch size = 16
INFO:tensorflow:  Num steps = 2137


In [0]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

test_input_fn = file_based_input_fn_builder(
    input_file=test_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
OUTPUT_DIR =  "/content/Output_Dir"
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

# tf.gfile.MkDir(OUTPUT_DIR)

# checkpoint_name = BERT_GCS_DIR + 'test.ckpt'
checkpoint_name = BERT_INIT_CHKPNT
OUTPUT_DIR_PER_MODEL = OUTPUT_DIR + "/arabert/"
print(OUTPUT_DIR_PER_MODEL)

In [0]:


if tf.gfile.Exists(OUTPUT_DIR_PER_MODEL):
  pass
else:
  tf.gfile.MkDir(OUTPUT_DIR_PER_MODEL)

num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_eval_steps = int(len(eval_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_steps_per_epoch = int(num_train_steps/BATCH_SIZE)

print("num train steps: {}".format(num_train_steps))
print("num warmup steps: {}".format(num_warmup_steps))
print("num_steps_per_epoch: {}".format(num_steps_per_epoch))

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels=len(LABEL_COLUMNS),
  init_checkpoint=checkpoint_name,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

run_config = tf.estimator.RunConfig(
    keep_checkpoint_max=100,
    model_dir=OUTPUT_DIR_PER_MODEL,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=200)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

print(f'Beginning Training!')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

print(f'Beginning Evaluation!')
eval_model_files = tf.gfile.Glob(os.path.join(OUTPUT_DIR_PER_MODEL,'*index'))

for eval_checkpoint in tqdm(sorted(eval_model_files,key=lambda x: int(x[0:-6].split('-')[-1]))):
# eval_checkpoint = eval_model_files[3]
  estimator.evaluate(input_fn=eval_input_fn, steps=int(num_eval_steps/BATCH_SIZE),checkpoint_path=eval_checkpoint[0:-6])

### Evaluating

In [0]:
import numpy as np
from sklearn.metrics import f1_score

offensive_true  = np.asarray(eval_["offensive"], dtype=np.int32)
hatespeech_true = np.asarray(eval_["hate_speech"], dtype=np.int32)

def get_f1(preds, offensive_true, hatespeech_true, f1_averaging):
  model_predictions = list()
  for i in preds:
    model_predictions.append(i)

  hate_speech_output = list()
  offense_output = list()
  for instance in model_predictions:
    hate_speech_output.append(list(instance['probabilities_hate']))
    offense_output.append(list(instance['probabilities_offense']))
  hate_speech_output = np.asarray(hate_speech_output)
  offense_output = np.asarray(offense_output)

  hate_speech_output = np.round(hate_speech_output).astype(np.int32)
  offense_output = np.round(offense_output).astype(np.int32)

  # print('not_offensive f1 score:', f1_score(offensive_true_one_hot[:, 0], offense_output[:, 0]))
  # print('not_hate_speech f1 score:', f1_score(hatespeech_true_one_hot[:, 0], hate_speech_output[:, 0]))
  # print('offensive f1 score:', f1_score(offensive_true_one_hot[:, 1], offense_output[:, 1]))
  # print('hate_speech f1 score:', f1_score(hatespeech_true_one_hot[:, 1], hate_speech_output[:, 1]))
  offensive_true_one_hot = np.zeros((offensive_true.size, offensive_true.max()+1))
  offensive_true_one_hot[np.arange(offensive_true.size), offensive_true] = 1
  hatespeech_true_one_hot = np.zeros((hatespeech_true.size, hatespeech_true.max()+1))
  hatespeech_true_one_hot[np.arange(hatespeech_true.size), hatespeech_true] = 1

  from prettytable import PrettyTable
    
  tab = PrettyTable()
  print('\n')
  tab.field_names = ["","not_offensive f1", 'not_hate_speech f1', 'offensive f1', 'hate_speech f1']
  tab.add_row(["values", np.round(f1_score(offensive_true_one_hot[:, 0], offense_output[:, 0], average= f1_averaging),4),
               np.round(f1_score(hatespeech_true_one_hot[:, 0], hate_speech_output[:, 0], average= f1_averaging),4),
               np.round(f1_score(offensive_true_one_hot[:, 1], offense_output[:, 1], average= f1_averaging),4), 
               np.round(f1_score(hatespeech_true_one_hot[:, 1], hate_speech_output[:, 1], average= f1_averaging),4)])
  print(tab)


  f1_not_hate_speech = f1_score(offensive_true_one_hot[:, 0], offense_output[:, 0])
  f1_not_hate_speech = f1_score(hatespeech_true_one_hot[:, 0], hate_speech_output[:, 0])
  f1_offensive = f1_score(offensive_true_one_hot[:, 1], offense_output[:, 1])
  f1_hate_speech = f1_score(hatespeech_true_one_hot[:, 1], hate_speech_output[:, 1])

  return f1_not_hate_speech, f1_not_hate_speech, f1_offensive, f1_hate_speech

In [0]:
# offensive_labels = x_val.iloc[:,1]
# hate_labels = x_val.iloc[:,2]
eval_model_files = tf.gfile.Glob(os.path.join(OUTPUT_DIR_PER_MODEL,'*index'))
for eval_checkpoint in tqdm(sorted(eval_model_files,key=lambda x: int(x[0:-6].split('-')[-1]))):
  
  result = estimator.predict(eval_input_fn, checkpoint_path=eval_checkpoint[0:-6])


  f1_not_hate_speech, f1_not_hate_speech, f1_offensive, f1_hate_speech = get_f1(result, offensive_true, hatespeech_true, 'macro')

  0%|          | 0/1 [00:00<?, ?it/s]

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 256)
INFO:tensorflow:  name = input_mask, shape = (?, 256)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?, 2)
INFO

100%|██████████| 1/1 [00:36<00:00, 36.97s/it]



+--------+------------------+--------------------+--------------+----------------+
|        | not_offensive f1 | not_hate_speech f1 | offensive f1 | hate_speech f1 |
+--------+------------------+--------------------+--------------+----------------+
| values |      0.9015      |       0.8341       |    0.9015    |     0.8341     |
+--------+------------------+--------------------+--------------+----------------+


#### Saving Best Model

In [0]:
eval_checkpoint = eval_model_files[7]
checkpoint_path = eval_checkpoint[0:-6]
# result = estimator.predict(eval_input_fn, checkpoint_path = eval_checkpoint[0:-6])

In [0]:
!cp -r /content/Output_Dir/arabert/model.ckpt-1200.index /content/best_model/
!cp -r /content/Output_Dir/arabert/model.ckpt-1200.meta /content/best_model/
!cp -r /content/Output_Dir/arabert/graph.pbtxt /content/best_model/
!cp -r /content/Output_Dir/arabert/events.out.tfevents.1582011537.9f8209d16eb2 /content/best_model/
!cp -r /content/Output_Dir/arabert/checkpoint /content/best_model/
!cp -r /content/Output_Dir/arabert/model.ckpt-1200.data-00000-of-00001 /content/best_model/

In [0]:
!zip -r best_model.zip /content/best_model

updating: content/best_model/ (stored 0%)
  adding: content/best_model/events.out.tfevents.1582011537.9f8209d16eb2 (deflated 92%)
  adding: content/best_model/checkpoint (deflated 84%)
  adding: content/best_model/model.ckpt-1200.meta (deflated 92%)
  adding: content/best_model/graph.pbtxt (deflated 97%)
  adding: content/best_model/model.ckpt-1200.index (deflated 69%)
  adding: content/best_model/model.ckpt-1200.data-00000-of-00001 (deflated 25%)


In [0]:
!cp  /content/best_model.zip /content/drive/'My Drive'/best_MTL_model/

In [0]:
result = estimator.predict(test_input_fn, checkpoint_path=eval_checkpoint[0:-6])

### Predicting on eval and test sets

In [0]:
def get_predictions(path_to_input_text, predictions, filename, true_labels, is_test_data):
  label_list = ['0','1']
  label_list_2=  ['0', '1']
  output_predict_file = "/content/test_results.tsv"
  res = []
  f = open(path_to_input_text, 'r', encoding='utf-8')
  lines = f.readlines()
  str1 = ''
  with tf.gfile.GFile(output_predict_file, "w") as writer:
    tf.logging.info("***** Predict results *****")

    for i, preds in enumerate(predictions):
      prediction_1 = preds["probabilities_offense"]
      prediction_2 = preds["probabilities_hate"]
      output_line = "\t".join(
          str(class_probability) for class_probability in prediction_1) + "\n"
      writer.write(output_line)

      prediction_1 = prediction_1.tolist()
      prediction_2 = prediction_2.tolist()
      pre_label_1 = label_list[prediction_1.index(max(prediction_1))]
      pre_label_2 = label_list_2[prediction_2.index(max(prediction_2))]
      # print(i)
      li = lines[i].strip().split('\t')
      # print(li)
      if is_test_data:
        str1 += li[0] + '\t' + pre_label_1 + '\t' + pre_label_2 + '\n'
      
      else: 
        str1 += li[0] + '\t' + pre_label_1 + '\t'+ true_labels.iloc[i,0] + '\t' + pre_label_2 + '\t' + true_labels.iloc[i,1] + '\n'

    

    with open('result.txt', 'w', encoding='utf-8') as f:
      f.write(str1)

    preds = pd.read_csv("/content/result.txt",sep="\t",header=None)
    if is_test_data:
      preds.columns =  ["text","offensive","hate_speech", ]
    else:
      
      preds.columns =  ["text","offensive","true_off","hate_speech", "true_hate"]

    preds.to_excel('{}.xlsx'.format(filename))

    return preds

In [0]:
eval_model_files

In [0]:
# preds = get_predictions('/content/OSACT2020-sharedTask-test-tweets.txt', result, 'test_preds')
eval_checkpoint = eval_model_files[7]
result_eval = estimator.predict(input_fn = eval_input_fn, checkpoint_path=eval_checkpoint[0:-6])
val_labels = x_train_w_eval.iloc[:,1:]
# train_labels = x_train.iloc[:,1:]
preds_eval = get_predictions('/content/data/OSACT2020-sharedTask-dev.txt', result_eval, 'eval_preds_1400', val_labels, is_test_data=True)


In [0]:
def get_predictions_final( predictions):
  label_list = ['NOT_OFF','OFF ']
  label_list_2=  ['NOT_HS', 'HS']
  output_predict_file = "/content/test_results.tsv"
  res = []
  str1 = ''
  str2 = ''
  with tf.gfile.GFile(output_predict_file, "w") as writer:
    tf.logging.info("***** Predict results *****")

    for i, preds in enumerate(predictions):
      prediction_1 = preds["probabilities_offense"]
      prediction_2 = preds["probabilities_hate"]
      output_line = "\t".join(
          str(class_probability) for class_probability in prediction_1) + "\n"
      writer.write(output_line)

      prediction_1 = prediction_1.tolist()
      prediction_2 = prediction_2.tolist()
      pre_label_1 = label_list[prediction_1.index(max(prediction_1))]
      pre_label_2 = label_list_2[prediction_2.index(max(prediction_2))]
      
      str1 +=  pre_label_1 + '\n'
      str2 +=  pre_label_2 + '\n'
    

    with open('result_off.txt', 'w', encoding='utf-8') as f:
      f.write(str1)
    
    with open('result_HS.txt', 'w', encoding='utf-8') as f:
      f.write(str2)


    return preds

In [0]:
x_test.shape

(2000, 3)

In [0]:
# preds = get_predictions('/content/OSACT2020-sharedTask-test-tweets.txt', result, 'test_preds')
eval_model_files = tf.gfile.Glob(os.path.join(OUTPUT_DIR_PER_MODEL,'*index'))
eval_checkpoint = eval_model_files[7]
result_eval = estimator.predict(input_fn = eval_input_fn, checkpoint_path=eval_checkpoint[0:-6])

# train_labels = x_train.iloc[:,1:]
preds_eval = get_predictions_final( result_eval )


In [0]:
!cp eval_preds.xlsx /content/drive/'My Drive'/

In [0]:
label_list = ['non-offense','offense']
label_list_2=  ['non-hate', 'hate']
output_predict_file = "/content/test_results.tsv"
res = []
f = open('/content/OSACT2020-sharedTask-test-tweets.txt', 'r', encoding='utf-8')
lines = f.readlines()
str1 = ''
with tf.gfile.GFile(output_predict_file, "w") as writer:
  tf.logging.info("***** Predict results *****")

  for i, predictions in enumerate(result):
    prediction = predictions["probabilities_offense"]
    prediction2 = predictions["probabilities_hate"]
    output_line = "\t".join(
        str(class_probability) for class_probability in prediction) + "\n"
    writer.write(output_line)

    prediction = prediction.tolist()
    prediction2 = prediction2.tolist()
    pre_label = label_list[prediction.index(max(prediction))]
    pre_label2 = label_list_2[prediction2.index(max(prediction2))]

    li = lines[i].strip().split('\t')
    str1 += li[0] + '\t' + pre_label + '\t' + pre_label2 + '\n'

  

  with open('result.txt', 'w', encoding='utf-8') as f:
    f.write(str1)

